In [3]:
# 강의 https://www.youtube.com/watch?v=tQUtBR3K1TI
#https://colab.research.google.com/drive/1S3jKF6Jofvl48fHUFqQwS-0iWMgl-pWl?usp=sharing#scrollTo=pWiMRQC1Kikl
!pip install chromadb tiktoken transformers sentence_transformers langchain pypdf


🕒 현재 시간: 2024-11-28 18:01:48
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached tokenizers-0.20.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.4.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 4.3 MB/s eta 0:00:00a 0:00:01
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 5.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.0 MB/s eta 0:00:00a 0:00:01
   

In [4]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

🕒 현재 시간: 2024-11-28 18:03:03
⏳ 실행 시간: 2.37초


In [11]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

import os.path        
print(os.path.abspath("../test-data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf"))

loader = PyPDFLoader("../test-data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function = tiktoken_len)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

docsearch = Chroma.from_documents(texts, hf)

🕒 현재 시간: 2024-11-28 18:08:17
/home/stlim/projects/samples/customAIService/test-data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf
⏳ 실행 시간: 8.73초


In [26]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOllama
from langchain_ollama.llms import OllamaLLM

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
model_name = "EEVE-Korean-Q5_K_M"
llm = ChatOllama(model=model_name
                 , temperature=0
                 , streaming=True
                 , callbacks=[StreamingStdOutCallbackHandler()]
                 )


🕒 현재 시간: 2024-11-28 18:17:17
⏳ 실행 시간: 0.00초


In [27]:


qa = RetrievalQA.from_chain_type(llm = llm,
                                 chain_type = "stuff",
                                 retriever = docsearch.as_retriever(
                                    search_type="mmr",
                                    search_kwargs={'k':3, 'fetch_k': 10}),
                                 return_source_documents = True)

query = "혁신성장 정책금융에 대해서 설명해줘"
result = qa(query)

🕒 현재 시간: 2024-11-28 18:17:20
혁신성장 정책금융은 건강한 혁신산업 생태계를 조성하기 위해 기업 성장에 필요한 자금을 지원하는 것을 목적으로 합니다. 이는 혁신성장에 대한 정의가 구체화된 정책금융 가이드라인을 바탕으로 운영되며, 9대 테마-46개 분야-296개 품목으로 구성되어 있습니다.

정책금융기관은 혁신성장 정책금융 제도를 시행한 이후 매년 공급 규모를 늘려왔으며, 미래 혁신성장 분야의 글로벌 경쟁력 확보를 위한 금융지원을 지속하고 있습니다. 2017년 240,787억 원에서 연평균 37.2% 증가하여 2021년에는 854,338억 원으로 확대되었습니다. 또한 정책금융 공급 건수도 같은 기간 동안 28,187건에서 71,369건으로 증가하였습니다.

ICT 산업의 경우 차세대 무선통신미디어, 능동형컴퓨팅(정보통신 테마), 차세대 반도체(전기전자 테마) 및 객체탐지(센서측정 테마) 기술분야에 정책금융이 집중되고 있습니다. 2017년 말 대비 2021년 말까지 정보통신 분야에서는 연평균 공급액 증가율이 37.4%, 전기전자 분야는 45.4%, 센서측정 분야는 51.4%로 나타나고 있습니다.

혁신성장 정책금융의 효과를 더욱 향상시키기 위해서는 ICIT 산업 내 주요 기술분야에 대한 가이드라인을 지속적으로 개선해야 합니다. 또한 미래 성장성은 유망하지만 단기 수익 창출이 어려운 산업을 지원하기 위해 민관주도의 역동적 금융으로 혁신성장 금융지원 영역을 확대하는 것이 필요합니다.⏳ 실행 시간: 44.68초


In [36]:
print(result)
print(type (result) )
print([  item.metadata for item in result['source_documents'] ])

🕒 현재 시간: 2024-11-28 18:23:20
{'query': '혁신성장 정책금융에 대해서 설명해줘', 'result': '혁신성장 정책금융은 건강한 혁신산업 생태계를 조성하기 위해 기업 성장에 필요한 자금을 지원하는 것을 목적으로 합니다. 이는 혁신성장에 대한 정의가 구체화된 정책금융 가이드라인을 바탕으로 운영되며, 9대 테마-46개 분야-296개 품목으로 구성되어 있습니다.\n\n정책금융기관은 혁신성장 정책금융 제도를 시행한 이후 매년 공급 규모를 늘려왔으며, 미래 혁신성장 분야의 글로벌 경쟁력 확보를 위한 금융지원을 지속하고 있습니다. 2017년 240,787억 원에서 연평균 37.2% 증가하여 2021년에는 854,338억 원으로 확대되었습니다. 또한 정책금융 공급 건수도 같은 기간 동안 28,187건에서 71,369건으로 증가하였습니다.\n\nICT 산업의 경우 차세대 무선통신미디어, 능동형컴퓨팅(정보통신 테마), 차세대 반도체(전기전자 테마) 및 객체탐지(센서측정 테마) 기술분야에 정책금융이 집중되고 있습니다. 2017년 말 대비 2021년 말까지 정보통신 분야에서는 연평균 공급액 증가율이 37.4%, 전기전자 분야는 45.4%, 센서측정 분야는 51.4%로 나타나고 있습니다.\n\n혁신성장 정책금융의 효과를 더욱 향상시키기 위해서는 ICIT 산업 내 주요 기술분야에 대한 가이드라인을 지속적으로 개선해야 합니다. 또한 미래 성장성은 유망하지만 단기 수익 창출이 어려운 산업을 지원하기 위해 민관주도의 역동적 금융으로 혁신성장 금융지원 영역을 확대하는 것이 필요합니다.', 'source_documents': [Document(metadata={'page': 2, 'source': '../test-data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf'}, page_content='1. 들어가며▶혁신성장 정책금융기관은 건강한 혁신산업 생태계를 조성하기 위해 기업 성장에 필요한 자금을 지원하는 혁신성장 정책금융 제도를 시행하고 있음￮혁신